In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)
    return image, cleaned

def find_bubbles(preprocessed_image):
    contours, _ = cv2.findContours(
        preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    bubbles = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = 4 * np.pi * area / (perimeter * perimeter)
        else:
            circularity = 0
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h if h > 0 else 0
        min_area = 100
        max_area = 5000
        if (min_area < area < max_area and 
            circularity > 0.7 and 
            0.8 < aspect_ratio < 1.2):
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                bubbles.append({
                    "contour": contour,
                    "centroid": (cx, cy),
                    "area": area,
                    "filled": None,
                    "option": None
                })
    return bubbles

def organize_bubbles_into_rows(bubbles, max_vertical_distance=50):
    if not bubbles:
        return []
    bubbles_sorted = sorted(bubbles, key=lambda b: b["centroid"][1])
    rows = []
    current_row = [bubbles_sorted[0]]
    for bubble in bubbles_sorted[1:]:
        if abs(bubble["centroid"][1] - current_row[-1]["centroid"][1]) < max_vertical_distance:
            current_row.append(bubble)
        else:
            rows.append(sorted(current_row, key=lambda b: b["centroid"][0]))
            current_row = [bubble]
    if current_row:
        rows.append(sorted(current_row, key=lambda b: b["centroid"][0]))
    return rows

def determine_filled_bubbles(image, rows):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for row_idx, row in enumerate(rows):
        fill_percentages = []
        for bubble in row:
            mask = np.zeros(gray.shape, dtype=np.uint8)
            cv2.drawContours(mask, [bubble["contour"]], -1, 255, -1)
            mean_value = cv2.mean(gray, mask=mask)[0]
            fill_percentage = 100 - (mean_value / 255 * 100)
            bubble["fill_percentage"] = fill_percentage
            fill_percentages.append(fill_percentage)
        if len(row) >= 2:
            if fill_percentages:
                max_fill_idx = np.argmax(fill_percentages)
                threshold = 30
                if fill_percentages[max_fill_idx] > threshold:
                    for i, bubble in enumerate(row):
                        if i == max_fill_idx:
                            bubble["filled"] = True
                            options = ["A", "B", "C", "D"]
                            if i < len(options):
                                bubble["option"] = options[i]
                        else:
                            bubble["filled"] = False
    return rows

def visualize_results(image, rows):
    result = image.copy()
    for row_idx, row in enumerate(rows):
        for bubble in row:
            color = (0, 0, 255)
            if bubble["filled"] == True:
                color = (0, 255, 0)
            cv2.drawContours(result, [bubble["contour"]], -1, color, 2)
            if bubble["option"]:
                cv2.putText(
                    result, 
                    bubble["option"], 
                    (bubble["centroid"][0] - 10, bubble["centroid"][1] - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2
                )
    plt.figure(figsize=(12, 10))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Bubbles')
    plt.tight_layout()
    plt.show()
    cv2.imwrite('result.jpg', result)
    return result

def extract_answers(rows):
    answers = []
    for row_idx, row in enumerate(rows):
        answer = None
        for bubble in row:
            if bubble["filled"] and bubble["option"]:
                answer = bubble["option"]
                break
        answers.append({
            "row": row_idx + 1,
            "answer": answer
        })
    return answers

def process_bubble_sheet(image_path):
    original, preprocessed = preprocess_image(image_path)
    bubbles = find_bubbles(preprocessed)
    rows = organize_bubbles_into_rows(bubbles)
    rows = determine_filled_bubbles(original, rows)
    result_image = visualize_results(original, rows)
    answers = extract_answers(rows)
    return result_image, answers

def main():
    image_paths = ['correct_mcq_sheet.png', 'student_mcq_sheet.png']
    for i, image_path in enumerate(image_paths):
        print(f"Processing image {i+1}: {image_path}")
        try:
            result_image, answers = process_bubble_sheet(image_path)
            print(f"\nResults for {image_path}:")
            for answer in answers:
                if answer["answer"]:
                    print(f"Row {answer['row']}: {answer['answer']}")
                else:
                    print(f"Row {answer['row']}: No answer detected")
            print("\n" + "-"*50 + "\n")
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

if __name__ == "__main__":
    main()
